In [ ]:
from google.colab import files
files.upload()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("default")

In [ ]:
trades = pd.read_csv("historical_data.csv")
sentiment = pd.read_csv("fear_greed_index.csv")

trades.head()
sentiment.head()

In [ ]:
trades.info()
sentiment.info()

In [ ]:
print(sentiment.columns)
print(trades.columns)

In [ ]:
sentiment['Date'] = pd.to_datetime(sentiment['date'], errors='coerce')


In [ ]:
sentiment.rename(columns={'classification': 'Classification'}, inplace=True)

In [ ]:
trades['time'] = pd.to_datetime(trades['Timestamp'], unit='ms', errors='coerce')

In [ ]:
sentiment['Date'] = sentiment['Date'].dt.date
trades['Date'] = trades['time'].dt.date

In [ ]:
sentiment[['date', 'Date']].head()
trades[['Timestamp', 'time', 'Date']].head()

In [ ]:
sentiment.info()
trades.info()

In [ ]:
sentiment['Date'] = pd.to_datetime(sentiment['Date'])
trades['time'] = pd.to_datetime(trades['time'])

In [ ]:
trades['Date'] = trades['time'].dt.date
sentiment['Date'] = sentiment['Date'].dt.date

In [ ]:
merged = pd.merge(trades, sentiment, on='Date', how='left')
merged.head()

In [ ]:
merged['abs_size_usd'] = merged['Size USD'].abs()
merged['is_profitable'] = merged['Closed PnL'] > 0

In [ ]:
merged['risk_score'] = merged['abs_size_usd'] * merged['Leverage'] \
    if 'Leverage' in merged.columns else merged['abs_size_usd']

In [ ]:
merged[['Size USD', 'abs_size_usd', 'Closed PnL', 'is_profitable']].head()

In [ ]:
import os

os.makedirs("outputs", exist_ok=True)
os.makedirs("csv_files", exist_ok=True)

In [ ]:
sns.countplot(data=merged, x='Classification')
plt.title("Trade Count by Market Sentiment")
plt.savefig("outputs/trades_by_sentiment.png", bbox_inches="tight")
plt.show()

Insight: Traders are more active during Greed phases and significantly more cautious during Fear periods.

In [ ]:
sns.boxplot(data=merged, x='Classification', y='Closed PnL')
plt.title("PnL Distribution by Market Sentiment")
plt.savefig("outputs/pnl_by_sentiment.png", bbox_inches="tight")
plt.show()

Insight: Profitability is more volatile during Greed phases, while Fear periods show relatively controlled PnL outcomes.

In [ ]:
sns.boxplot(data=merged, x='Classification', y='Size USD')
plt.title("Trade Size (USD) by Market Sentiment")
plt.savefig("outputs/size_usd_by_sentiment.png", bbox_inches="tight")
plt.show()

Insight: Trade sizes are larger during Greed periods, indicating increased risk-taking compared to more conservative sizing during Fear phases.

In [ ]:
win_rate = merged.groupby('Classification')['is_profitable'].mean().reset_index()

sns.barplot(data=win_rate, x='Classification', y='is_profitable')
plt.title("Win Rate by Market Sentiment")
plt.ylabel("Win Rate")
plt.savefig("outputs/win_rate_by_sentiment.png", bbox_inches="tight")
plt.show()

Insight: Despite higher risk-taking during Greed phases, win rates do not improve significantly, suggesting potential overconfidence.

In [ ]:
!ls outputs


In [ ]:
!zip -r outputs.zip outputs
files.download("outputs.zip")